In [64]:
import pandas as pd
import numpy as np

melbourne = pd.read_csv("temperature_Melbourne_celsius.csv", delimiter=";")
oslo = pd.read_csv("temperature_Oslo_celsius.csv", delimiter=";")
quito = pd.read_csv("temperature_Quito_celsius.csv", delimiter=";")

# Ejercicio 1A

In [65]:
# Asegurar que la columna de temperatura sea la correcta
columna_temperatura = "AvgTemperature"  # Cambia esto según el nombre de la columna en los archivos

# Calcular la media y desviación estándar
media_melbourne = melbourne[columna_temperatura].mean()
media_oslo = oslo[columna_temperatura].mean()
media_quito = quito[columna_temperatura].mean()

desvio_melbourne = melbourne[columna_temperatura].std()
desvio_oslo = oslo[columna_temperatura].std()
desvio_quito = quito[columna_temperatura].std()

# Mostrar los resultados con formato mejorado
print(f"Media de Melbourne: {media_melbourne:.2f}°C")
print(f"Media de Oslo: {media_oslo:.2f}°C")
print(f"Media de Quito: {media_quito:.2f}°C")

print(f"Desvío estándar de Melbourne: {desvio_melbourne:.2f}°C")
print(f"Desvío estándar de Oslo: {desvio_oslo:.2f}°C")
print(f"Desvío estándar de Quito: {desvio_quito:.2f}°C")

Media de Melbourne: 17.53°C
Media de Oslo: 4.61°C
Media de Quito: 13.60°C
Desvío estándar de Melbourne: 6.48°C
Desvío estándar de Oslo: 9.46°C
Desvío estándar de Quito: 1.30°C


### 1 -B

In [66]:
from scipy.signal import correlate
import numpy as np

x = oslo[columna_temperatura].values
y = melbourne[columna_temperatura].values

corr = correlate(x, y, mode='full')
print(f"Correlación entre Oslo y Melbourne: {corr.max():.2f}")

x = oslo[columna_temperatura].values
y = quito[columna_temperatura].values

corr = correlate(x, y, mode='full')
print(f"Correlación entre Oslo y Quito: {corr.max():.2f}")

x = quito[columna_temperatura].values
y = melbourne[columna_temperatura].values

corr = correlate(x, y, mode='full')
print(f"Correlación entre quito y Melbourne: {corr.max():.2f}")


Correlación entre Oslo y Melbourne: 624455.00
Correlación entre Oslo y Quito: 376918.00
Correlación entre quito y Melbourne: 1393015.00


Analizando los resultados obtenidos observamos que entre oslo y cualquir otra ciudad no hay tremperatura climatica similar, en cambio entre Quito y Merburne hay una mayor correlacion por lo que hay una similitud, lo cual tiene sentido observando las medias.

## 2.
### Considerando los valores de temperatura t que componen cada señal Si, construir una nueva señal Ti  compuesta por una secuencia de símbolos discretos F, T o C, definidos según: 

In [67]:

def clasificar_temperatura(t):
    if t < 11:
        return 'F'
    elif t < 19:
        return 'T'
    else:
        return 'C'

for df in [oslo, quito, melbourne]:
    df['Clima'] = df[columna_temperatura].apply(clasificar_temperatura)



#### Parte 1

In [68]:
def contar_climas(df):
    conteo = df['Clima'].value_counts()
    f =  conteo.get('F', 0) 
    t = conteo.get('T', 0)
    c =  conteo.get('C', 0)
    return f, t, c

f_oslo, t_oslo, c_oslo = contar_climas(oslo)
f_quito, t_quito, c_quito = contar_climas(quito)
f_melbourne, t_melbourne, c_melbourne = contar_climas(melbourne)

def contar_transiciones(df, origen):
    """
    Cuenta cuántas veces después de 'origen' aparece cada letra ('F', 'T', 'C') en la columna 'Clima' del DataFrame.
    Retorna un diccionario con el conteo de transiciones.
    """
    clima = df['Clima'].values
    transiciones = {'F': 0, 'T': 0, 'C': 0}
    for i in range(len(clima) - 1):
        if clima[i] == origen:
            siguiente = clima[i + 1]
            if siguiente in transiciones:
                transiciones[siguiente] += 1
    return transiciones

def distribucion_probabilidad(df,f, t, c):
    matriz = [[0,0,0],
            [0,0,0],
            [0,0,0]]
    i = 0
    for l in ['F', 'T', 'C']:
        transiciones = contar_transiciones(df, l)
        matriz[i][0] = round(transiciones['F']/f, 3)
        matriz[i][1] = round(transiciones['T']/t, 3)
        matriz[i][2] = round(transiciones['C']/c, 3)
        i += 1
    return np.array(matriz)   

distribucion_oslo = distribucion_probabilidad(oslo, f_oslo, t_oslo, c_oslo)
distribucion_quito = distribucion_probabilidad(quito, f_quito, t_quito, c_quito)
distribucion_melbourne = distribucion_probabilidad(melbourne, f_melbourne, t_melbourne, c_melbourne)

print("Distribución de probabilidad de Oslo:")
print(distribucion_oslo)
print("--------------------------------------------")
print("Distribución de probabilidad de Quito:")
print(distribucion_quito)
print("--------------------------------------------")
print("Distribución de probabilidad de Melbourne:")
print(distribucion_melbourne)


Distribución de probabilidad de Oslo:
[[0.959 0.099 0.   ]
 [0.041 0.865 0.296]
 [0.    0.035 0.704]]
--------------------------------------------
Distribución de probabilidad de Quito:
[[0.054 0.006 0.   ]
 [0.946 0.993 1.   ]
 [0.    0.001 0.   ]]
--------------------------------------------
Distribución de probabilidad de Melbourne:
[[0.47  0.033 0.003]
 [0.495 0.836 0.144]
 [0.035 0.131 0.853]]


#### Parte 2

In [ ]:
def contarColumnas(df):
    conteo = df['Clima'].value_counts()
    f =  conteo.get('F', 0) 
    t = conteo.get('T', 0)
    c =  conteo.get('C', 0)
    return f+t+c

def distribucion_probabilistica_estacionaria(df):
    totalColumnas=contarColumnas(df)
    f,t,c = contar_climas(df)
    res = [f /totalColumnas, t / totalColumnas, c / totalColumnas]

    return np.array(res, dtype=np.float64)

distribucion_estacionaria_oslo = distribucion_probabilistica_estacionaria(oslo)
print(distribucion_estacionaria_oslo)

[0.68583162 0.28062971 0.03353867]


In [70]:
def matriz_acumulada(matriz):
    res = matriz.copy()
    for i in range(0,len(matriz)): #i recorre las filas
        for j in range(1, len(matriz[i])): # j recorre las columnas
            res[j][i] += res[j-1][i]
    return res

acumulada_oslo = matriz_acumulada(distribucion_oslo)
acumulada_quito = matriz_acumulada(distribucion_quito)
acumulada_melbourne = matriz_acumulada(distribucion_melbourne)

print("Matriz acumulada de quito:")
print(acumulada_quito)

Matriz acumulada de quito:
[[0.054 0.006 0.   ]
 [1.    0.999 1.   ]
 [1.    1.    1.   ]]


In [71]:
import random

def Media_recurrencia(simbolo, matriz):
    retornos= 0        # cantidad de retornos a si 
    media= 0          #media recurrencia actual    
    media_ant= -1  # media recurrencia anterior    
    t_actual= 0   
    s= simbolo
    while  not np.allclose(media, media_ant, atol=1e-3) or (t_actual<100) :
        s= Sig_dado_Ant (s, matriz)
        t_actual+=1
        if ( s == simbolo ): #hay retorno 
            retornos+=1
            media_ant = media
            media= t_actual/retornos
    return media

def Sig_dado_Ant(simbolo, matriz):
    r = random.random()
    if simbolo == "F":
        simbolo = 0
    elif simbolo == "T":
        simbolo = 1
    else:
        simbolo = 2

    for i in range(3):
        if ( r < matriz[i, simbolo]):
            if(i==0):
                return "F"
            elif(i==1):
                return "T"
            else:
                return "C"
"""
MR_F_oslo = Media_recurrencia("F", acumulada_oslo)
MR_T_oslo = Media_recurrencia("T", acumulada_oslo)
MR_C_oslo = Media_recurrencia("C", acumulada_oslo)
"""
MR_F_quito = Media_recurrencia("F", acumulada_quito)
MR_T_quito = Media_recurrencia("T", acumulada_quito)
MR_C_quito = Media_recurrencia("C", acumulada_quito)
"""
MR_F_melbourne = Media_recurrencia("F", acumulada_melbourne)
MR_T_melbourne = Media_recurrencia("T", acumulada_melbourne)
MR_C_melbourne = Media_recurrencia("C", acumulada_melbourne)

print(f"Media de recurrencia de F en Oslo: {MR_F_oslo:.2f}")
print(f"Media de recurrencia de T en Oslo: {MR_T_oslo:.2f}")
print(f"Media de recurrencia de C en Oslo: {MR_C_oslo:.2f}")
"""

print(f"Media de recurrencia de F en Quito: {MR_F_quito:.2f}")
print(f"Media de recurrencia de T en Quito: {MR_T_quito:.2f}")
print(f"Media de recurrencia de C en Quito: {MR_C_quito:.2f}")
"""
print("--------------------------------------------")
print(f"Media de recurrencia de F en Melbourne: {MR_F_melbourne:.2f}")
print(f"Media de recurrencia de T en Melbourne: {MR_T_melbourne:.2f}")
print(f"Media de recurrencia de C en Melbourne: {MR_C_melbourne:.2f}")
"""

Media de recurrencia de F en Quito: 164.33
Media de recurrencia de T en Quito: 1.00
Media de recurrencia de C en Quito: 968.94


'\nprint("--------------------------------------------")\nprint(f"Media de recurrencia de F en Melbourne: {MR_F_melbourne:.2f}")\nprint(f"Media de recurrencia de T en Melbourne: {MR_T_melbourne:.2f}")\nprint(f"Media de recurrencia de C en Melbourne: {MR_C_melbourne:.2f}")\n'

Calcular la entropía de cada fuente Ti :
sin memoria (orden 0), considerando símbolos individuales.
con memoria (orden 1), usando información sobre transiciones entre símbolos.



#### entropia sin memoria

In [72]:

def entropia(df):
    distribucion = distribucion_probabilistica_estacionaria(df)
    entropia = -np.sum(distribucion * np.log2(distribucion))  # Añadir un pequeño valor para evitar log(0)
    return entropia


for df in [oslo, quito, melbourne]:
    entopia = entropia(df)
    nombre = "Oslo" if df is oslo else "Quito" if df is quito else "Melbourne"

    print(f"Entropía de {nombre}: c")






Entropía de Oslo: c
Entropía de Quito: c
Entropía de Melbourne: c


#### la ciudad con mayor entropia es melbourne esto indica que es la ciudad menos predecible de las tres, asi mismo la ciudad con menor entropia es quito inidcando que su clima es altamente predicible, lo cual tiene sentido si vemos la media de recurrencia donde prevalece el clima templado por sobre los demas.

#### fuente con memoria

In [73]:

def entropia_matriz(matriz,df):
    estacionario = distribucion_probabilistica_estacionaria(df) 
    entropia = 0
    for i in range(matriz.shape[0]): 
        distribucion = matriz[:, i] # creo arreglo con la columna i
        entropiaColumna = -np.sum(distribucion * np.log2(distribucion + 1e-10))  # Añadir un pequeño valor para evitar log(0)
        entropia += entropiaColumna * estacionario[i]
    
    return entropia    

print(f"entropia de oslo con memoria 1 = {entropia_matriz(distribucion_oslo, oslo):.4f}")
print(f"entropia de quito con memoria 1 = {entropia_matriz(distribucion_quito, quito):.4f}")
print(f"entropia de melbourne con memoria 1 = {entropia_matriz(distribucion_melbourne, melbourne):.4f}")


entropia de oslo con memoria 1 = 0.3897
entropia de quito con memoria 1 = 0.0658
entropia de melbourne con memoria 1 = 0.7130


In [74]:

import heapq  # Módulo para manejar colas de prioridad (heaps)

class Nodo:
    def __init__(self, simbolo=None, probabilidad=None):
        self.simbolo = simbolo
        self.probabilidad = probabilidad
        self.izquierda = None
        self.derecha = None

    def __lt__(self, otro):
        # Define cómo comparar nodos según su probabilidad
        return self.probabilidad < otro.probabilidad

def construir_arbol_huffman(simbolos, probabilidades):
    # Crear una cola de prioridad con los símbolos y sus probabilidades
    cola = [Nodo(s, p) for s, p in zip(simbolos, probabilidades)]
    heapq.heapify(cola)
    print(simbolos)
    # Construir el árbol de Huffman
    while len(cola) > 1:
        # Sacar los dos nodos con menor probabilidad
        nodo_izq = heapq.heappop(cola)
        nodo_der = heapq.heappop(cola)
        # Crear un nuevo nodo combinando los dos hijos
        nodo_combinado = Nodo(probabilidad=nodo_izq.probabilidad + nodo_der.probabilidad)
        nodo_combinado.izquierda = nodo_izq
        nodo_combinado.derecha = nodo_der
        # Volver a insertar el nodo combinado en la cola
        heapq.heappush(cola, nodo_combinado)

    # Devolver la raíz del árbol
    return cola[0]

def generar_codigos_huffman(nodo, codigo="", codigos={}):
    if nodo is not None:
        if nodo.simbolo is not None:
            # Se llegó a una hoja: asignar el código
            codigos[nodo.simbolo] = codigo
        # Recorrer recursivamente los hijos izquierdo y derecho
        generar_codigos_huffman(nodo.izquierda, codigo + "0", codigos)
        generar_codigos_huffman(nodo.derecha, codigo + "1", codigos)

    return codigos


In [75]:
codificacionLongitud_1= []

for df in [oslo, quito, melbourne]:
    
    simbolos = ['F', 'T', 'C']
    probabilidades = distribucion_probabilistica_estacionaria(df)   # Probabilidades de los símbolos F, T y C

    # Construir el árbol de Huffman
    raiz = construir_arbol_huffman(simbolos, probabilidades)

    # Generar los códigos
    codigos_huffman = generar_codigos_huffman(raiz)
    
    # Mostrar los resultados
    for simbolo, codigo in codigos_huffman.items():
        nombre = "Oslo" if df is oslo else "Quito" if df is quito else "Melbourne"
        # Guardar símbolo, ciudad y código en una lista de tuplas
        codificacionLongitud_1.append((simbolo, nombre, codigo))
        print(f"Símbolo: {simbolo}, Ciudad: {nombre}, Código: {codigo}")

print(codificacionLongitud_1)            


['F', 'T', 'C']
Símbolo: C, Ciudad: Oslo, Código: 00
Símbolo: T, Ciudad: Oslo, Código: 01
Símbolo: F, Ciudad: Oslo, Código: 1
['F', 'T', 'C']
Símbolo: C, Ciudad: Quito, Código: 00
Símbolo: T, Ciudad: Quito, Código: 1
Símbolo: F, Ciudad: Quito, Código: 01
['F', 'T', 'C']
Símbolo: C, Ciudad: Melbourne, Código: 01
Símbolo: T, Ciudad: Melbourne, Código: 1
Símbolo: F, Ciudad: Melbourne, Código: 00
[('C', 'Oslo', '00'), ('T', 'Oslo', '01'), ('F', 'Oslo', '1'), ('C', 'Quito', '00'), ('T', 'Quito', '1'), ('F', 'Quito', '01'), ('C', 'Melbourne', '01'), ('T', 'Melbourne', '1'), ('F', 'Melbourne', '00')]


In [76]:
def extension_orden_dos(df,matriz):
    probabilidades = []
    estacionario = distribucion_probabilistica_estacionaria(df)  
    for i in range(matriz.shape[0]): 
        for j in range(matriz.shape[1]):
            probabilidades.append(matriz[i][j] * estacionario[i])
    return probabilidades             


In [77]:
def limpiar_simbolos(simbolos,probabilidades):
    it = 0
    while it < len(probabilidades):
        if probabilidades[it] == 0:
            simbolos.pop(it)
            probabilidades.pop(it)
        it+= 1
## repetimos codigo porque se bugea el jupyter y no reconoce las funciones definidas anteriormente

import heapq  # Módulo para manejar colas de prioridad (heaps)

class Nodo:
    def __init__(self, simbolo=None, probabilidad=None):
        self.simbolo = simbolo
        self.probabilidad = probabilidad
        self.izquierda = None
        self.derecha = None

    def __lt__(self, otro):
        # Define cómo comparar nodos según su probabilidad
        return self.probabilidad < otro.probabilidad

def construir_arbol_huffman(simbolos, probabilidades):
    # Crear una cola de prioridad con los símbolos y sus probabilidades
    cola = [Nodo(s, p) for s, p in zip(simbolos, probabilidades)]
    heapq.heapify(cola)
    # Construir el árbol de Huffman
    while len(cola) > 1:
        # Sacar los dos nodos con menor probabilidad
        nodo_izq = heapq.heappop(cola)
        nodo_der = heapq.heappop(cola)
        # Crear un nuevo nodo combinando los dos hijos
        nodo_combinado = Nodo(probabilidad=nodo_izq.probabilidad + nodo_der.probabilidad)
        nodo_combinado.izquierda = nodo_izq
        nodo_combinado.derecha = nodo_der
        # Volver a insertar el nodo combinado en la cola
        heapq.heappush(cola, nodo_combinado)

    # Devolver la raíz del árbol
    return cola[0]

def generar_codigos_huffman(nodo, codigo="", codigos={}):
    if nodo is not None:
        if nodo.simbolo is not None:
            # Se llegó a una hoja: asignar el código
            codigos[nodo.simbolo] = codigo
        # Recorrer recursivamente los hijos izquierdo y derecho
        generar_codigos_huffman(nodo.izquierda, codigo + "0", codigos)
        generar_codigos_huffman(nodo.derecha, codigo + "1", codigos)

    return codigos


for df in [oslo, quito, melbourne]:
    simbolos = ['FF', 'TF', 'CF', 'FT', 'TT', 'CT', 'FC', 'TC', 'CC']
    
    if df is oslo:
        matriz = distribucion_oslo
        nombre = "Oslo"
    elif df is quito:
        matriz = distribucion_quito
        nombre = "Quito"
    else:
        nombre = "Melbourne"
        matriz = distribucion_melbourne

    probabilidades = extension_orden_dos(df, matriz)
    
    limpiar_simbolos(simbolos, probabilidades)
    print(f"simbolos limpios: {simbolos}")
    
    # Construir el árbol de Huffman
    raiz = construir_arbol_huffman(simbolos, probabilidades)
    
    # Generar los códigos
    codigos_huffman = generar_codigos_huffman(raiz)

    # Mostrar los resultados
    for simbolo, codigo in codigos_huffman.items():
        print(f"Símbolo: {simbolo}, Ciudad: {nombre}, Código: {codigo}")
    

simbolos limpios: ['FF', 'TF', 'FT', 'TT', 'CT', 'TC', 'CC']
Símbolo: CT, Ciudad: Oslo, Código: 000
Símbolo: TC, Ciudad: Oslo, Código: 001000
Símbolo: FT, Ciudad: Oslo, Código: 001001
Símbolo: CC, Ciudad: Oslo, Código: 00101
Símbolo: TF, Ciudad: Oslo, Código: 0011
Símbolo: TT, Ciudad: Oslo, Código: 01
Símbolo: FF, Ciudad: Oslo, Código: 1
simbolos limpios: ['FF', 'TF', 'FT', 'TT', 'CT', 'TC']
Símbolo: CT, Ciudad: Quito, Código: 0
Símbolo: TC, Ciudad: Quito, Código: 10000
Símbolo: FT, Ciudad: Quito, Código: 101
Símbolo: CC, Ciudad: Quito, Código: 00101
Símbolo: TF, Ciudad: Quito, Código: 10001
Símbolo: TT, Ciudad: Quito, Código: 11
Símbolo: FF, Ciudad: Quito, Código: 1001
simbolos limpios: ['FF', 'TF', 'CF', 'FT', 'TT', 'CT', 'FC', 'TC', 'CC']
Símbolo: CT, Ciudad: Melbourne, Código: 1100
Símbolo: TC, Ciudad: Melbourne, Código: 11011
Símbolo: FT, Ciudad: Melbourne, Código: 111
Símbolo: CC, Ciudad: Melbourne, Código: 10
Símbolo: TF, Ciudad: Melbourne, Código: 11010101
Símbolo: TT, Ciudad: 

In [78]:

def aplicar_compresion(clima, ciudad):
    for simbolo, nombre, codigo in codificacionLongitud_1:
        if clima == simbolo and ciudad == nombre:
            return codigo
    return None  # Si no encuentra el código

for df in [oslo, quito, melbourne]:
    df_nuevo = df.copy()
    nombre_ciudad = "Oslo" if df is oslo else "Quito" if df is quito else "Melbourne"

    df_nuevo['codificacion'] = df['Clima'].apply(lambda clima: aplicar_compresion(clima, nombre_ciudad))
    df_nuevo = df_nuevo.drop(columns=['Clima', 'AvgTemperature'])
    # Guardar el nuevo DataFrame a un archivo CSV
    df_nuevo.to_csv(f"codificacion_{nombre_ciudad}.csv", index=False)

In [79]:
import os

def calcular_tasa_compresion(nombre_archivo_original, nombre_archivo_codificado, ciudad):
    size_original = os.path.getsize(nombre_archivo_original) / 1024  # KB
    size_codificado = os.path.getsize(nombre_archivo_codificado) / 1024  # KB
    tasa = round(size_original / size_codificado, 2)
    print(f"La tasa de compresión para el dataset de {ciudad} {size_original:.0f}kb antes y ahora {size_codificado:.0f}kb es de {tasa}")


calcular_tasa_compresion("temperature_Melbourne_celsius.csv", "codificacion_melbourne.csv", "Melbourne")
calcular_tasa_compresion("temperature_Quito_celsius.csv", "codificacion_quito.csv", "quito")
calcular_tasa_compresion("temperature_Oslo_celsius.csv", "codificacion_oslo.csv", "oslo")

La tasa de compresión para el dataset de Melbourne 34kb antes y ahora 20kb es de 1.72
La tasa de compresión para el dataset de quito 34kb antes y ahora 17kb es de 1.99
La tasa de compresión para el dataset de oslo 33kb antes y ahora 19kb es de 1.72


#### ej4


In [104]:
melbourneRiudoso = pd.read_csv("temperature_Melbourne_celsius_ruidoso.csv", delimiter=";")
melbourneRiudoso['Clima'] = melbourneRiudoso[columna_temperatura].apply(clasificar_temperatura)

canal = pd.crosstab(melbourne['Clima'],melbourneRiudoso['Clima'],  normalize='index')

# Reordena filas y columnas a ['F', 'C', 'T']
canal = canal.reindex(index=['F', 'C', 'T'], columns=['F', 'C', 'T'])
canal


Clima,F,C,T
Clima,,,
F,0.610000,0.000000,0.390000
C,0.000000,0.756978,0.243022
T,0.139669,0.185661,0.674670


#### Estabilidad en "C": El estado "C" tiene una alta probabilidad (75%) de mantenerse como "C", aunque un 25% de las veces se transforma en "T". No hay cambios hacia "F", lo que indica que el ruido no introduce transición hacia calor desde frío.

#### Ruido en "F": Aunque "F" se mantiene en el mismo estado el 69% de las veces, hay un 39% de probabilidad de que pase a "T". Es interesante notar que nunca cambia a "C", lo que sugiere que el ruido tiende a mover las temperaturas cálidas hacia templadas, pero no hacia frías.

#### Inestabilidad en "T": El estado "T" es el más afectado por el ruido, con una probabilidad del 67% de mantenerse en "T", pero con un 20% de transición hacia "C" y un 13% hacia "F". Esto sugiere que los valores templados sufren más fluctuaciones por el ruido, moviéndose tanto hacia frío como hacia calor.


In [105]:

#  cuenta P(X)
#  normalize=True hace que los conteos se conviertan en proporciones (probabilidades), reindex reordena las filas según el índice del canal
entrada_probs = melbourne['Clima'].value_counts(normalize=True).reindex(canal.index)

def entropia_fila(probs_fila):
    return -np.sum([p * np.log2(p) for p in probs_fila if p > 0])

# 4. Calcular H(Y|X)
ruido = sum(
    entrada_probs[x] * entropia_fila(canal.loc[x])
    for x in canal.index
)

print(f"Ruido del canal (H(Y|X)) = {ruido:.4f}")    




Ruido del canal (H(Y|X)) = 1.0236


In [107]:
salida_probs = melbourneRiudoso['Clima'].value_counts(normalize=True)

H_Y = entropia_fila(salida_probs)

informacion_mutua = H_Y - ruido
print(f"Información mutua I(X;Y) = {informacion_mutua:.4f}")

Información mutua I(X;Y) = 0.3260
